In [ ]:
# Notes - landscan will be the reference grid

In [ ]:
# !pip install rasterio
# !pip install pyproj

In [1]:
import os
import rasterio
from rasterio.windows import Window, from_bounds
from rasterio.warp import Resampling
from rasterio.vrt import WarpedVRT
from rasterio import shutil as rio_shutil

In [2]:
from pyproj import Transformer

# sanity check - coordinates to check
# first coord is in Yangon, second is rural, the rest are randomly selected from the survey
coords = [(96.199379, 16.871311), (96.103347, 18.613386), (97.233770, 24.260900), (95.419190, 17.681720), (96.163024, 16.774673), (95.404810, 16.286770)]

# given path to a tif file and a list of lon-lat coordinates
# this returns the values at those coordinates
def check_vals_coord(path, coord_lst):
    with rasterio.open(path) as rds:
        # convert coordinate to raster projection
        transformer = Transformer.from_crs(str(rds.crs), rds.crs, always_xy=True)
        pts = [transformer.transform(tup[0], tup[1]) for tup in coord_lst]
        # get value from grid
        vals = [list(rds.sample([e]))[0][0] for e in pts]
    return vals

In [5]:
dirpath = 'C:/Users/sun-joo.lee/Documents/0_git_repos/Myanmar_Survey_ML/data/geo/'
refile = 'landscan/global/landscan-global-2017.tif'
globpath = 'viirs/global/vcmcfg_avg_rade9h_'
clippath = 'viirs/clipped/vcmcfg_clip_'

In [8]:
# bounding box coordinates for myanmar
mm_bbox = [91.5, 9.0, 101.5, 29.0]

In [9]:
# get vrt options from reference (landscan) file
with rasterio.open(f'{dirpath}{refile}') as src:
    win = from_bounds(*mm_bbox, src.transform)
    win = Window(*[max(0, v) for v in win.flatten()])
    print(type(src.window_transform(win)))
    vrt_options = {
        "crs": src.crs.to_epsg(),
        "transform": src.window_transform(win),
        "width": int(win.width),
        "height": int(win.height),
    }
    
vrt_options.update({'resampling':Resampling.bilinear})
vrt_options

<class 'affine.Affine'>


{'crs': 4326,
 'transform': Affine(0.0083333333333333, 0.0, 91.5,
        0.0, -0.0083333333333333, 29.0),
 'width': 1200,
 'height': 2400,
 'resampling': <Resampling.bilinear: 1>}

In [10]:
link_dict = {
    '201701': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201701/vcmcfg/SVDNB_npp_20170101-20170131_global_vcmcfg_v10_c201702241223.avg_rade9h.tif',
    '201702': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201702/vcmcfg/SVDNB_npp_20170201-20170228_global_vcmcfg_v10_c201703012030.avg_rade9h.tif',
    '201703': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201703/vcmcfg/SVDNB_npp_20170301-20170331_global_vcmcfg_v10_c201705020851.avg_rade9h.tif',
    '201704': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201704/vcmcfg/SVDNB_npp_20170401-20170430_global_vcmcfg_v10_c201705011300.avg_rade9h.tif',
    '201705': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201705/vcmcfg/SVDNB_npp_20170501-20170531_global_vcmcfg_v10_c201706021500.avg_rade9h.tif',
    '201706': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201706/vcmcfg/SVDNB_npp_20170601-20170630_global_vcmcfg_v10_c201707021700.avg_rade9h.tif',
    '201707': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201707/vcmcfg/SVDNB_npp_20170701-20170731_global_vcmcfg_v10_c201708061230.avg_rade9h.tif',
    '201708': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201708/vcmcfg/SVDNB_npp_20170801-20170831_global_vcmcfg_v10_c201709051000.avg_rade9h.tif',
    '201709': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201709/vcmcfg/SVDNB_npp_20170901-20170930_global_vcmcfg_v10_c201710041620.avg_rade9h.tif',
    '201710': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201710/vcmcfg/SVDNB_npp_20171001-20171031_global_vcmcfg_v10_c201711021230.avg_rade9h.tif',
    '201711': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201711/vcmcfg/SVDNB_npp_20171101-20171130_global_vcmcfg_v10_c201712040930.avg_rade9h.tif',
    '201712': 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2017/201712/vcmcfg/SVDNB_npp_20171201-20171231_global_vcmcfg_v10_c201801021747.avg_rade9h.tif'
}

In [12]:
import requests

# download chunk size
CHUNK = 2 ** 10

for year_month, url in link_dict.items():
    
    # save the global file
    globfile = f'{dirpath}{globpath}{year_month}.tif'
    with open(globfile, 'wb') as f:
        ret = requests.get(url, stream=True)
        for data in ret.iter_content(CHUNK):
            f.write(data)
    
    # warp and save the clipped file
    # https://rasterio.readthedocs.io/en/latest/topics/virtual-warping.html
    clipfile = f'{dirpath}{clippath}{year_month}.tif'
    with rasterio.open(globfile) as src:
        with WarpedVRT(src, **vrt_options) as vrt:
            data = vrt.read()
            for _, window in vrt.block_windows():
                data = vrt.read(window=window)
            rio_shutil.copy(vrt, clipfile, driver='GTiff')
    
    # sanity check
    print(year_month)
    print(check_vals_coord(globfile, coords))    
    print(check_vals_coord(clipfile, coords))    
    print()

201701
[13.09, 0.01, 5.76, 0.06, 47.43, 2.07]
[13.8376465, 0.016876183, 4.044367, 0.07811758, 45.285015, 2.0959654]

201702
[13.32, 0.11, 3.82, 0.11, 41.6, 2.54]
[13.749897, 0.11188203, 3.099089, 0.14874202, 47.524075, 2.5841362]

201703
[12.58, 0.3, 3.8, 0.22, 39.4, 2.44]
[13.661392, 0.28436726, 2.8865807, 0.26124838, 45.775574, 2.3671215]

201704
[11.61, 0.32, 3.47, 0.39, 40.97, 1.7]
[11.958814, 0.31499025, 2.6608102, 0.41312444, 42.551296, 1.7195928]

201705
[11.98, 0.31, 4.17, 0.38, 35.16, 1.41]
[11.8645, 0.2781384, 3.5413263, 0.3456169, 41.97414, 1.5383265]

201706
[6.53, 0.31, 2.17, 0.37, 43.88, 0.99]
[7.0851307, 0.28811744, 2.245393, 0.31935504, 43.24536, 0.9369193]

201707
[4.76, 0.23, 2.63, 0.28, 0.0, 0.0]
[4.487892, 0.10058674, 2.3456128, 0.23687153, 0.0, 0.033163562]

201708
[7.54, 0.24, 2.26, 0.2, 36.85, 1.65]
[7.8517184, 0.25812885, 2.542585, 0.2981151, 43.42656, 1.4858137]

201709
[10.66, 0.54, 2.85, 0.4, 71.43, 1.45]
[13.154704, 0.51310635, 2.5821352, 0.42937326, 74.7175